In [1]:

from data_utils import export_data_for_inspection
if __name__ == "__main__":

    print("--- 开始执行数据导出任务 ---")
    export_data_for_inspection(column_to_inspect='roa_yearly')

    print("\n" + "="*50 + "\n")
    print("--- 所有导出任务已完成 ---")

from data_utils import export_wide_panel

if __name__ == "__main__":
    print("--- 开始执行“长表转宽表”导出任务 ---")
    export_wide_panel(value_column='roa_yearly')
    print("\n" + "="*50 + "\n")
    print("--- 所有面板数据导出任务已完成 ---")


--- 开始执行数据导出任务 ---
--- 准备导出用于检查的 'roa_yearly' 原始数据 ---
数据加载成功！
数据已准备和排序完毕。

--- 正在导出数据到Excel文件 ---

[成功] 数据已成功导出到: /Users/alan-hopiy/Documents/个人研究/基本面量化/roa_yearly_source_data_for_inspection.xlsx
请打开此文件进行检查。


--- 所有导出任务已完成 ---
--- 开始执行“长表转宽表”导出任务 ---
--- 准备将 'roa_yearly' 数据转换为宽表 ---
数据加载成功！

去重前的数据行数: 173325
去重后的数据行数: 104549

数据清洗与去重完成。

正在执行 pivot_table 操作...
宽表创建成功！

--- 正在导出宽表到Excel文件 ---

[成功] 已将宽表格式的数据导出到: /Users/alan-hopiy/Documents/个人研究/基本面量化/roa_yearly_wide_format.xlsx


--- 所有面板数据导出任务已完成 ---


In [3]:
import pandas as pd
import numpy as np
import os # 引入os模块来处理文件路径

# ==============================================================================
#  步骤 0: 从您指定的Excel文件路径加载数据
# ==============================================================================
print("--- 步骤 0: 从Excel文件加载准备好的毛利率宽表数据 ---")

file_path = '/Users/alan-hopiy/Documents/个人研究/基本面量化/不用不用，不用改那个计算周期完全按照原本的来哈喽酷狗_wide_format.xlsx'

try:
    df_gross_margin = pd.read_excel(file_path, index_col=0)
    print(f"成功从路径 '{file_path}' 加载数据！")
    print("数据预览：")
    print(df_gross_margin.head())

except FileNotFoundError:
    print(f"[错误] 文件未找到！请检查路径是否正确: {file_path}")
    df_gross_margin = pd.DataFrame()
except Exception as e:
    print(f"读取Excel文件时出错: {e}")
    df_gross_margin = pd.DataFrame()

# ==============================================================================
#  数据预处理与因子计算流程
# ==============================================================================

if not df_gross_margin.empty:
    print("\n--- 开始执行数据预处理和因子计算 ---")
    
    # --- 步骤 1: 数据充分性筛选 ---
    print("\n--- 步骤 1: 数据充分性筛选 ---")
    valid_counts = df_gross_margin.notna().sum(axis=1)
    MIN_PERIODS = 12
    df_filtered = df_gross_margin[valid_counts >= MIN_PERIODS]
    print(f"原始公司数量: {len(df_gross_margin)}")
    print(f"筛选后剩余公司数量: {len(df_filtered)}")

    if not df_filtered.empty:
        # --- 步骤 2: 极端值处理 (缩尾) ---
        print("\n--- 步骤 2: 极端值处理 (缩尾) ---")
        lower_bound = df_filtered.stack().quantile(0.01)
        upper_bound = df_filtered.stack().quantile(0.99)
        df_winsorized = df_filtered.clip(lower=lower_bound, upper=upper_bound)
        print(f"已对数据在 [{lower_bound:.2f}, {upper_bound:.2f}] 范围内进行缩尾处理。")

        # --- 步骤 3: 计算处理后数据的平均值 ---
        print("\n--- 步骤 3: 计算处理后数据的平均值 ---")
        average_gm = df_winsorized.mean(axis=1)
        
        # --- 步骤 4: Z-Score 标准化 ---
        print("\n--- 步骤 4: Z-Score 标准化 ---")
        z_scores_gm = (average_gm - average_gm.mean()) / average_gm.std()
        print("Z-Score 计算完成。")

        # ==============================================================================
        #  新增部分：步骤 5 - 导出最终结果到Excel
        # ==============================================================================
        print("\n--- 步骤 5: 导出Z-score因子值到Excel文件 ---")
        
        # 定义输出的文件夹路径和完整文件路径
        output_dir = '/Users/alan-hopiy/Documents/个人研究/基本面量化/'
        output_filename = 'factor_zscore_roa.xlsx'
        output_path = os.path.join(output_dir, output_filename)
        
        try:
            # 确保输出文件夹存在
            os.makedirs(output_dir, exist_ok=True)
            
            # 将z_scores_gm这个Series转换为DataFrame并导出
            # z_scores_gm.to_frame()会把Series变成一列的DataFrame
            # 我们给这一列命名为 'z_score_gm'
            z_scores_gm.to_frame(name='z_score_gm').to_excel(output_path, index=True)
            
            print(f"\n[成功] 因子已成功导出到: {output_path}")
            
        except Exception as e:
            print(f"\n[失败] 文件导出失败: {e}")
        # ==============================================================================

    else:
        print("经过数据充分性筛选后，没有剩余的公司可供处理。")
else:
    print("数据加载失败或数据为空，未执行后续计算。")

--- 步骤 0: 从Excel文件加载准备好的毛利率宽表数据 ---
成功从路径 '/Users/alan-hopiy/Documents/个人研究/基本面量化/roa_yearly_wide_format.xlsx' 加载数据！
数据预览：
           2020-03-31  2020-06-30  2020-09-30  2020-12-31  2021-03-31  \
ts_code                                                                 
000001.SZ      0.8472      0.6740      0.7208      0.6881      0.8964   
000002.SZ      0.5604      2.1074      2.2625      3.2952      0.5312   
000004.SZ      0.9624      0.2812      4.8063      4.0542      1.0944   
000006.SZ      3.6200      3.0194      4.5185      5.7915      7.2216   
000007.SZ    -17.6956    -11.6340    -12.0265    -27.4001     -1.4248   

           2021-06-30  2021-09-30  2021-12-31  2022-03-31  2022-06-30  \
ts_code                                                                 
000001.SZ      0.7652      0.8335      0.7739      1.0240      0.8808   
000002.SZ      1.6916      1.7101      1.9995      0.5736      1.8966   
000004.SZ      0.7862      2.8413    -38.0366     -4.7272     -6.6910   


In [5]:
# file: build_roa_yearly_growth_factor_final.py

import pandas as pd
import numpy as np
import config
import os

def calculate_growth_with_penalty(row, target_years):
    """
    【最终版】成长率计算函数。
    - 沿用最初基于季度面板的计算逻辑。
    - 只有当期初和期末指标都为正时，才计算CAGR。
    - 否则，只要涉及负值或零，就给予-10的惩罚性低分。
    """
    series = row.dropna()
    
    # ==============================================================================
    #  关键点：完全按照您原有的季度逻辑 (年数 * 4)
    # ==============================================================================
    target_period = target_years * 4 
    
    if len(series) <= target_period: 
        return np.nan
        
    latest_roa = series.iloc[-1]
    base_roa = np.nan
    
    lookback_periods = [target_period, target_period - 1, target_period + 1, target_period - 2]
    actual_period_in_quarters = np.nan
    for period in lookback_periods:
        if 0 < period < len(series):
            potential_base = row.iloc[-(period + 1)]
            if pd.notna(potential_base):
                base_roa = potential_base
                actual_period_in_quarters = period
                break
            
    if pd.isna(base_roa):
        return np.nan
        
    if base_roa > 0 and latest_roa > 0:
        # 这里的年数计算也保持原样
        num_years = actual_period_in_quarters / 4.0
        return (latest_roa / base_roa) ** (1 / num_years) - 1
    else:
        # 惩罚分数已修改为 -10
        return -10

# ==============================================================================
#  步骤 1: 加载 roa_yearly 面板数据
# ==============================================================================
print("--- 步骤 1: 加载预处理好的 roa_yearly 面板数据 ---")
input_dir = config.BASE_OUTPUT_DIR
input_path = os.path.join(input_dir, "roa_yearly_wide_format.xlsx") 
try:
    roa_yearly_panel = pd.read_excel(input_path, index_col=0)
    print(f"成功加载文件，数据面板形状为: {roa_yearly_panel.shape}")
except FileNotFoundError:
    raise SystemExit(f"[错误] 输入文件未找到: {input_path}。请先运行之前的脚本生成此文件。")

# ==============================================================================
#  步骤 2: 计算多周期成长率
# ==============================================================================
print("\n--- 步骤 2: 计算带质量惩罚的多周期成长率 ---")
results_df = pd.DataFrame(index=roa_yearly_panel.index)
cagr_cols = []
for years in [2, 3, 4]:
    print(f"正在计算 {years}年期 CAGR...")
    col_name = f'roa_yearly_cagr_{years}yr_penalized'
    cagr_cols.append(col_name)
    results_df[col_name] = roa_yearly_panel.apply(calculate_growth_with_penalty, axis=1, target_years=years)

# ==============================================================================
#  步骤 2.5: 执行“一票否决”式强力惩罚
# ==============================================================================
print("\n--- 步骤 2.5: 执行“一票否决”式强力惩罚 ---")
PENALTY_FLAG = -10
FINAL_PENALTY_VALUE = -10
is_penalized_mask = (results_df[cagr_cols] == PENALTY_FLAG).any(axis=1)
results_df.loc[is_penalized_mask, cagr_cols] = FINAL_PENALTY_VALUE
penalized_count = is_penalized_mask.sum()
print(f"检测到 {penalized_count} 家公司存在历史瑕疵，已对其所有成长率得分应用强力惩罚。")

# ==============================================================================
#  步骤 3: 独立进行Z-score打分
# ==============================================================================
print("\n--- 步骤 3: 独立进行Z-score打分 ---")
zscore_cols = []
for years in [2, 3, 4]:
    cagr_col = f'roa_yearly_cagr_{years}yr_penalized'
    zscore_col = f'roa_yearly_zscore_{years}yr_penalized'
    zscore_cols.append(zscore_col)
    
    mean = results_df[cagr_col].mean()
    std = results_df[cagr_col].std()
    
    results_df[zscore_col] = (results_df[cagr_col] - mean) / std
    print(f"完成 {zscore_col} 的计算。")
    
# ==============================================================================
#  步骤 4 到 8 (与之前版本完全一致，仅修改了名称)
# ==============================================================================
# --- 步骤 4: 筛选 ---
print("\n--- 步骤 4: 基于有效得分数量，筛选高质量公司 ---")
MIN_VALID_SCORES = 2 
valid_score_counts = results_df[zscore_cols].count(axis=1)
results_df_filtered = results_df[valid_score_counts >= MIN_VALID_SCORES].copy()
print(f"筛选前公司数量: {len(results_df)}")
print(f"筛选后公司数量: {len(results_df_filtered)}")

# --- 步骤 5: 填充缺失 ---
print("\n--- 步骤 5: 对缺失的Z-score填充0值 (作为中性惩罚) ---")
results_df_filtered[zscore_cols] = results_df_filtered[zscore_cols].fillna(0)
print("缺失Z-score填充完成。")

# --- 步骤 6: 合成 ---
print("\n--- 步骤 6: 合成得分并进行最终标准化 ---")
composite_avg = results_df_filtered[zscore_cols].mean(axis=1)
final_mean = composite_avg.mean()
final_std = composite_avg.std()
final_score_col = 'composite_roa_yearly_growth_score_final'
results_df_filtered[final_score_col] = (composite_avg - final_mean) / final_std
print("最终得分合成与标准化完成。")

# --- 步骤 7: 预览 ---
results_df_filtered.sort_values(by=final_score_col, ascending=False, inplace=True)
print("\n--- 最终ROA(Yearly)成长性打分排名预览 (前10名) ---")
display_cols = [col for col in results_df_filtered.columns if 'cagr' not in col]
print(results_df_filtered[display_cols].head(10))

# --- 步骤 8: 导出 ---
print("\n--- 步骤 8: 导出最终的因子得分文件 ---")
output_filename = "final_composite_roa_yearly_growth_scores.xlsx"
output_path = os.path.join(input_dir, output_filename)
try:
    results_df_filtered.to_excel(output_path, index=True)
    print(f"\n[成功] 已将最终的ROA(Yearly)成长分导出到: {output_path}")
except Exception as e:
    print(f"\n[失败] 文件导出失败: {e}")

--- 步骤 1: 加载预处理好的 roa_yearly 面板数据 ---
成功加载文件，数据面板形状为: (5412, 20)

--- 步骤 2: 计算带质量惩罚的多周期成长率 ---
正在计算 2年期 CAGR...
正在计算 3年期 CAGR...
正在计算 4年期 CAGR...

--- 步骤 2.5: 执行“一票否决”式强力惩罚 ---
检测到 1977 家公司存在历史瑕疵，已对其所有成长率得分应用强力惩罚。

--- 步骤 3: 独立进行Z-score打分 ---
完成 roa_yearly_zscore_2yr_penalized 的计算。
完成 roa_yearly_zscore_3yr_penalized 的计算。
完成 roa_yearly_zscore_4yr_penalized 的计算。

--- 步骤 4: 基于有效得分数量，筛选高质量公司 ---
筛选前公司数量: 5412
筛选后公司数量: 5338

--- 步骤 5: 对缺失的Z-score填充0值 (作为中性惩罚) ---
缺失Z-score填充完成。

--- 步骤 6: 合成得分并进行最终标准化 ---
最终得分合成与标准化完成。

--- 最终ROA(Yearly)成长性打分排名预览 (前10名) ---
           roa_yearly_zscore_2yr_penalized  roa_yearly_zscore_3yr_penalized  \
ts_code                                                                       
600223.SH                         1.883234                         0.981385   
300641.SZ                         1.458000                         1.013882   
601500.SH                         1.428640                         1.172702   
600605.SH                         1.792739    